In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.widgets.text("storage_name", "adlssmartdatajuanfinal")
dbutils.widgets.text("container", "raw")
dbutils.widgets.text("catalogo", "catalog_dev")
dbutils.widgets.text("esquema", "bronze")

In [0]:
storage_name = dbutils.widgets.get("storage_name")
container = dbutils.widgets.get("container")
catalogo = dbutils.widgets.get("catalogo")
esquema = dbutils.widgets.get("esquema")



In [0]:
fecha_actual_archivo = spark.sql("""
    SELECT date_format(
        from_utc_timestamp(current_timestamp(), 'America/Bogota'), 
        'yyyyMMdd'
    ) AS fecha
""").collect()[0]["fecha"]

# Mostrar el resultado
print(fecha_actual_archivo)

20251110


In [0]:
# Ruta de maestro y detalle haciendo referencia al archivo que termine con la fecha del día de ejecución

ruta_maestro = f"abfss://{container}@{storage_name}.dfs.core.windows.net/Maestro_Horas_"+fecha_actual_archivo+".csv"
ruta_bitacora = f"abfss://{container}@{storage_name}.dfs.core.windows.net/Bitacora_Registros_"+fecha_actual_archivo+".csv"

In [0]:
maestro_schema = StructType(fields=[StructField("Zona", StringType(), False),
                                     StructField("Criticidad", StringType(), False),
                                     StructField("Tiempo_Espera_Horas", IntegerType(), False),
                                     StructField("Fecha_Ejecucion", TimestampType(), False)               
])


bitacora_schema = StructType(fields=[StructField("Cod_incidente", StringType(), False),
                                     StructField("Zona", StringType(), False),
                                     StructField("Estado", StringType(), False),
                                     StructField("Hora_Asignacion", StringType(), True),
                                     StructField("Tipo_Vehiculo", StringType(), True),
                                     StructField("Numero_Tecnicos", IntegerType(), True),
                                     StructField("Equipo_Afectado", StringType(), True),
                                     StructField("Tipo_Falla", StringType(), True),
                                     StructField("Prioridad", IntegerType(), False),
                                     StructField("Lider_Tecnico", StringType(), True),
                                     StructField("Fecha_Ejecucion", TimestampType(), False),
])

In [0]:
df_maestro = spark.read\
    .option('header', True)\
    .option('sep', ';')\
    .schema(maestro_schema)\
    .csv(ruta_maestro)


df_bitacora = spark.read\
    .option('header', True)\
    .option('sep', ';')\
    .schema(bitacora_schema)\
    .csv(ruta_bitacora)

In [0]:
# Ajustar Fecha_Ejecucion a hora colombiana
df_maestro_final = df_maestro.withColumn(
    "Fecha_Ejecucion", 
    from_utc_timestamp(current_timestamp(), "America/Bogota")
)

df_bitacora_final = df_bitacora.withColumn(
    "Fecha_Ejecucion", 
    from_utc_timestamp(current_timestamp(), "America/Bogota")
)

# Formatear Fecha_Ejecucion como string legible
df_maestro_final = df_maestro_final.withColumn(
    "Fecha_Ejecucion", 
    date_format("Fecha_Ejecucion", "yyyy-MM-dd HH:mm:ss")
)

df_bitacora_final = df_bitacora_final.withColumn(
    "Fecha_Ejecucion", 
    date_format("Fecha_Ejecucion", "yyyy-MM-dd HH:mm:ss")
)

# Convertir Hora_Asignacion desde formato string a timestamp
df_bitacora_final = df_bitacora_final.withColumn(
    "Hora_Asignacion", 
    to_timestamp("Hora_Asignacion", "dd/MM/yyyy HH:mm:ss")
)

# Ajustar Hora_Asignacion a hora colombiana y formatear
df_bitacora_final = df_bitacora_final.withColumn(
    "Hora_Asignacion", 
    date_format(from_utc_timestamp("Hora_Asignacion", "America/Bogota"), "yyyy-MM-dd HH:mm:ss")
)


In [0]:
# Obtener fecha actual en hora Colombia
fecha_colombia = spark.sql("SELECT to_date(from_utc_timestamp(current_timestamp(), 'America/Bogota')) AS fecha").collect()[0]["fecha"]

# Ejecutar DELETE con fecha colombiana
spark.sql(f"""
DELETE FROM catalog_dev.bronze.maestro_horas
WHERE to_date(from_utc_timestamp(Fecha_Ejecucion, 'America/Bogota')) = DATE('{fecha_colombia}')
""")

spark.sql(f"""
DELETE FROM catalog_dev.bronze.bitacora_incidentes
WHERE to_date(from_utc_timestamp(Fecha_Ejecucion, 'America/Bogota')) = DATE('{fecha_colombia}')
""")




DataFrame[num_affected_rows: bigint]

In [0]:
# Obtener los schemas de las tablas de destino
target_schema_maestro = spark.table(f"{catalogo}.{esquema}.Maestro_horas").schema
target_schema_bitacora = spark.table(f"{catalogo}.{esquema}.Bitacora_incidentes").schema

# Casteo de datos de las columnas del dataframe para que coincidadn con el schema de la tabla final
df_maestro_casted = df_maestro_final.select(
    [col(field.name).cast(field.dataType).alias(field.name) for field in target_schema_maestro]
)
df_bitacora_casted = df_bitacora_final.select(
    [col(field.name).cast(field.dataType).alias(field.name) for field in target_schema_bitacora]
)

df_maestro_casted.write.mode("append").saveAsTable(f"{catalogo}.{esquema}.Maestro_horas")
df_bitacora_casted.write.mode("append").saveAsTable(f"{catalogo}.{esquema}.Bitacora_incidentes")